<div dir="rtl" align="center">
    <h1>
        تمرین اول درس پردازش زبان‌های طبیعی
    </h1>
    <h3>
        گردآورندگان:<br/>
        ساحل مس‌فروش، سروش تابش، درنا دهقانی
    </h3>
    
</div>

<br/>
<br/>


<div dir="rtl">
    ما در این تمرین ترک "استخراج پرسش و پاسخ" را انتخاب کردیم. در این تمرین قصد داریم از یک جمله فارسی سوالاتی با توجه به روابط علت و معلولی، نقش کلمات در جملات، ارتباط بین واژگان و ... استخراج کنیم. هر سوال به شکل یک دیکشنری پایتون نمایش داده می‌شود که key سوال و value پاسخ به سوال است.
</div>

<div dir="rtl">
    <h2> 1. تمیز کردن ورودی </h2>
    در این بخش، گام‌های ابتدایی برای مراحل آتی را برمی‌داریم. هر جمله که به عنوان ورودی ما داده می‌شود، باید تا حد مناسبی تمیز و ساختاریافته شود تا به عنوان ورودی بخش‌های بعد معتبر باشد. کارهایی مانند نرمال‌سازی و ... در این بخش انجام می‌گردد. از توابع تعریف شده در کتابخانه hazm استفاده می‌کنیم.
</div>

In [7]:
from hazm import *

def sentence_clean(sentence: str):
    normalizer = Normalizer()
    return normalizer.normalize(sentence)

sentence = ''
print(sentence_clean(sentence))

<div dir="rtl">
    <h2> 2. استخراج ساختار سوالات </h2>
    در این مرحله، به یافتن کلمات کلیدی در جملاتی که امکان استخراج سوالات از آنها وجود دارد پرداختیم. در یک Google Sheet لیست این کلمات، انواع نحوه به کارگیری آنها در جملات، مثال‌هایی از هر یک و هم‌چنین ساختار سوال و جواب و مثال از آنها را مشخص کردیم. 
    هم‌چنین مثال‌هایی از جملات را به عنوان داده‌ی تست یافتیم و سوال‌هایی از آنها استخراج کردیم.
    <a href="https://docs.google.com/spreadsheets/d/1qsCfRVr5RgXYW2qEVbrP3dq7-l3wNngjPAsXrHN89DQ/edit#gid=0" target="_top">فهرست کلمات کلیدی و جملات تست</a>
</div>